# Randomized Retail GL Generator
_Procured by Analytical Ants LLC_

This repository hosts a Python script that generates a randomized, yet logical, general ledger dataset tailored for retail businesses. It's a perfect tool for financial modeling training, educational purposes, and for anyone who needs a quick mock-up of a general ledger without diving deep into manual data entry.

For more information, please see full README: https://github.com/AnalyticalAnts/Randomized-Retail-GL-Generator/blob/main/README.md

In [2]:
import random
import csv
from datetime import datetime, timedelta

# Set seed for replicable results
random.seed(12345)

# Define accounts and their descriptions
accounts = {
    101: "Cash",
    102: "Accounts Receivable",
    103: "Inventory",
    201: "Accounts Payable",
    202: "Loan Payable",
    301: "Owner's Equity",
    401: "Sales Revenue",
    501: "Cost of Goods Sold",
    502: "Rent Expense",
    503: "Salary Expense"
}

# Start date for transactions two years ago from now
start_date = datetime.now() - timedelta(days=730)

# Initialize the running totals for each account
account_totals = {account: 0 for account in accounts}

def update_totals(entries):
    for entry in entries:
        if len(entry) == 6:  # Ensure we don't process description lines
            account = entry[2]
            dr = entry[4]
            cr = entry[5]
            # Asset and Expense accounts
            if account in [101, 102, 103, 501, 502, 503]:
                account_totals[account] += dr - cr
            # Liability, Owner's Equity, and Revenue accounts
            elif account in [201, 202, 301, 401]:
                account_totals[account] += cr - dr

def check_and_correct_equation(index, curr_date):
    assets = sum(account_totals[account] for account in [101, 102, 103])
    liabilities = sum(account_totals[account] for account in [201, 202])
    equity = account_totals[301]

    discrepancy = assets - (liabilities + equity)

    # If there's a discrepancy, correct by adjusting Owner's Equity
    if discrepancy != 0:
        correction_entry = (index, curr_date.strftime('%Y-%m-%d'), 301, accounts[301], 0 if discrepancy > 0 else -discrepancy, 0 if discrepancy < 0 else discrepancy)
        return [correction_entry]
    return []

def generate_ledger_entry(index, curr_date):
    entries = []
    trans_date = curr_date.strftime('%Y-%m-%d')
    
    batch_type_descriptions = {
        "owner_contribution": ["Owner's capital injection", "Owner adding funds", "Capital added by owner"],
        "sales": ["Weekly sales", "Revenue from sales", "Sales revenue recorded"],
        "purchase_inventory": ["Inventory purchase", "Bought goods for resale", "Added to inventory stock"],
        "rent_and_salaries": ["Monthly fixed costs", "Rent and salaries payment", "Operational costs for the month"],
        "loan_received": ["Taking out a loan", "Loan received from bank", "Received bank loan"],
        "purchase": ["Goods purchased", "Added to inventory", "Restocking products"],
        "sale": ["Items sold", "Sales completed", "Goods exchanged for money"],
        "expenses": ["Regular expenses", "Monthly bills paid", "Operational costs deducted"],
        "loan_activity": ["Loan activity", "Bank interactions", "Loan account adjustments"],
        "payable_payment": ["Debt payment", "Paid off a creditor", "Reduced payable amount"]
    }

    batch_type = random.choice(list(batch_type_descriptions.keys()))

    # Initial Capital Injection
    if index == 1:
        capital = round(random.uniform(50000, 70000), 2)
        entries.append((index, trans_date, 101, accounts[101], capital, 0))  # Debit cash
        entries.append((index, trans_date, 301, accounts[301], 0, capital))  # Credit Owner's Equity

    else:
        batch_type = random.choice(["purchase", "sale", "expenses", "loan_activity", "payable_payment"])

        # # 1. Buying inventory
        # if batch_type == "purchase":
        #     amount = round(random.uniform(500, 5000), 2)
        #     entries.append((index, trans_date, 103, accounts[103], amount, 0))  # Debit inventory
        #     entries.append((index, trans_date, 201, accounts[201], 0, amount))  # Credit accounts payable

        # # 2. Making sales
        # elif batch_type == "sale":
        #     sales_amount = round(random.uniform(600, 6000), 2)  # Sales amount slightly higher than inventory cost for profit
        #     cogs = sales_amount * 0.85  # Assuming a profit margin of 15%
        #     entries.append((index, trans_date, 401, accounts[401], 0, sales_amount))  # Credit sales
        #     entries.append((index, trans_date, 501, accounts[501], cogs, 0))  # Debit COGS
        #     entries.append((index, trans_date, 103, accounts[103], 0, cogs))  # Credit Inventory
        #     payment_type = random.choice(["cash", "credit"])
        #     if payment_type == "cash":
        #         entries.append((index, trans_date, 101, accounts[101], sales_amount, 0))  # Debit cash
        #     else:
        #         entries.append((index, trans_date, 102, accounts[102], sales_amount, 0))  # Debit accounts receivable

        # 1. Buying inventory
        if batch_type == "purchase":
            amount = round(random.uniform(500, 5000), 2)
            entries.append((index, trans_date, 103, accounts[103], amount, 0))  # Debit inventory
            entries.append((index, trans_date, 201, accounts[201], 0, amount))  # Credit accounts payable

        # 2. Making sales
        elif batch_type == "sale":
            sales_amount = round(random.uniform(600, 6000), 2)  # Sales amount slightly higher than inventory cost for profit
            cogs = sales_amount * 0.85  # Assuming a profit margin of 15%
            # Ensure there's enough inventory to sell
            if account_totals[103] >= cogs:
                entries.append((index, trans_date, 401, accounts[401], 0, sales_amount))  # Credit sales
                entries.append((index, trans_date, 501, accounts[501], cogs, 0))  # Debit COGS
                entries.append((index, trans_date, 103, accounts[103], 0, cogs))  # Credit Inventory
                payment_type = random.choice(["cash", "credit"])
                if payment_type == "cash":
                    entries.append((index, trans_date, 101, accounts[101], sales_amount, 0))  # Debit cash
                else:
                    entries.append((index, trans_date, 102, accounts[102], sales_amount, 0))  # Debit accounts receivable

        # 3. Paying expenses
        elif batch_type == "expenses":
            rent = round(random.uniform(1000, 3000), 2)
            salary = round(random.uniform(3000, 7000), 2)
            entries.append((index, trans_date, 502, accounts[502], rent, 0))    # Debit rent expense
            entries.append((index, trans_date, 503, accounts[503], salary, 0))  # Debit salary expense
            entries.append((index, trans_date, 101, accounts[101], 0, rent + salary))  # Credit cash

        # 4. Loan activities
        elif batch_type == "loan_activity":
            loan_activity = random.choice(["borrow", "repay"])
            loan_amount = round(random.uniform(5000, 10000), 2)
            if loan_activity == "borrow":
                entries.append((index, trans_date, 101, accounts[101], loan_amount, 0))  # Debit cash
                entries.append((index, trans_date, 202, accounts[202], 0, loan_amount))  # Credit loan payable
            else:
                entries.append((index, trans_date, 101, accounts[101], 0, loan_amount))  # Credit cash
                entries.append((index, trans_date, 202, accounts[202], loan_amount, 0))  # Debit loan payable

        # # 5. Paying off accounts payable
        # elif batch_type == "payable_payment":
        #     payable_amount = round(random.uniform(500, 2000), 2)
        #     entries.append((index, trans_date, 201, accounts[201], payable_amount, 0))  # Debit accounts payable
        #     entries.append((index, trans_date, 101, accounts[101], 0, payable_amount))  # Credit cash


        # 5. Paying off accounts payable
        elif batch_type == "payable_payment":
            # Ensure payable amount doesn't exceed current payables
            max_payable = min(account_totals[201], round(random.uniform(500, 2000), 2))
            if max_payable > 0:
                entries.append((index, trans_date, 201, accounts[201], max_payable, 0))  # Debit accounts payable
                entries.append((index, trans_date, 101, accounts[101], 0, max_payable))  # Credit cash


    # Append description line
        desc_line = (index, "Description:", random.choice(batch_type_descriptions[batch_type]), "", "", "", "")
        entries.append(desc_line)
    
    return entries, curr_date + timedelta(days=random.randint(1, 10))

# Generate the dataset
dataset = []
index = 1
curr_date = start_date
while index <= 50:
    entries, curr_date = generate_ledger_entry(index, curr_date)
    dataset.extend(entries)
    
    update_totals(entries)
    corrections = check_and_correct_equation(index, curr_date)
    dataset.extend(corrections)
    update_totals(corrections)
    
    index += 1

# Write to a CSV file
try:
    with open('Synthetic_Retail_GL_Data.csv', 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(["BATCH", "DATE", "ACCOUNT", "ACCOUNT DESCRIPTION", "DR", "CR"])  # header row
        writer.writerows(dataset)
    print("Dataset generated and saved to Synthetic_Retail_GL_Data.csv!")
except PermissionError:
    print("Error: The file 'Synthetic_Retail_GL_Data.csv' is open or you do not have permission to write to it. Please close it and try again.")

Dataset generated and saved to Synthetic_Retail_GL_Data.csv!


In [3]:
def initial_budget():
    """Generate an initial budget for the start of the period"""
    # These values are just exemplary. They can be adjusted based on industry averages or other assumptions.
    return {
        101: 10000,   # Cash
        102: 5000,    # Accounts Receivable
        103: 2500,    # Inventory
        201: -2000,   # Accounts Payable (negative as it's a liability)
        202: -5000,   # Loan Payable
        301: 15000,   # Owner's Equity
        401: 8000,    # Sales Revenue
        501: 2500,    # Cost of Goods Sold
        502: 1000,    # Rent Expense
        503: 1500     # Salary Expense
    }

def rebudget(previous_budget, last_6_months_actuals):
    """Generate a new budget based on previous budget and last 6 months' actuals"""
    # Calculate monthly actuals from the last 6 months
    monthly_actuals = {account: total / 6 for account, total in last_6_months_actuals.items()}
    
    # New budget is a weighted average of the previous budget and recent actuals
    # Weights: 70% to previous budget and 30% to actuals (these can be adjusted)
    new_budget = {}
    for account in previous_budget:
        new_budget[account] = 0.7 * previous_budget[account] + 0.3 * monthly_actuals[account]
        
        # Apply a 2% growth rate (can be adjusted)
        new_budget[account] *= 1.02
    
    # Ensuring cash doesn't go negative
    if new_budget[101] < 0:
        discrepancy = abs(new_budget[101])
        new_budget[101] = 0
        # Adjust discrepancy in Owner's Equity
        new_budget[301] -= discrepancy
    
    # Multiply by 6 to get the budget for the next 6 months
    new_budget = {account: value * 6 for account, value in new_budget.items()}
    
    return new_budget

# Start with an initial budget
budgets = [initial_budget()]

# Simulate actuals and budgets over the two years
for i in range(4):  # 4 iterations for 2 years (every 6 months)
    # For simplicity, let's assume the actuals in the script represent a 6 months period
    # Therefore, the current account_totals represent the last 6 months' actuals
    new_budget = rebudget(budgets[-1], account_totals)
    budgets.append(new_budget)

    # Reset account_totals to simulate next 6 months
    account_totals = {account: 0 for account in accounts}

budgets


[{101: 10000,
  102: 5000,
  103: 2500,
  201: -2000,
  202: -5000,
  301: 15000,
  401: 8000,
  501: 2500,
  502: 1000,
  503: 1500},
 {101: 60927.18263999999,
  102: 31526.38134,
  103: 43.4852010000012,
  201: -8798.099760000001,
  202: -8671.23522,
  301: 69268.384161,
  401: 50160.25134,
  501: 24215.013638999997,
  502: 8949.21786,
  503: 18919.670939999996},
 {101: 261012.05042975995,
  102: 135059.01766055997,
  103: 186.29060108400512,
  201: -37691.059371840005,
  202: -37147.571682480004,
  301: 296745.75774572394,
  401: 214886.51674056,
  501: 103737.11842947597,
  502: 38338.44931224,
  503: 81051.87030695999},
 {101: 1118175.6240410917,
  102: 578592.8316578388,
  103: 798.0689350438779,
  201: -161468.49834896257,
  202: -159140.1970877443,
  301: 1271258.8261826811,
  401: 920573.837716559,
  501: 444409.81535187503,
  502: 164241.91685363615,
  503: 347226.21239501657},
 {101: 4790264.373392037,
  102: 2478691.690822181,
  103: 3418.9273177279724,
  201: -691731.04692

In [4]:
def format_budget_as_gl(budget, start_date):
    """Format the budget to resemble a general ledger"""
    formatted_budget = []
    # We'll split the budget equally over the 6 months
    monthly_budget = {account: value / 6 for account, value in budget.items()}
    
    for month in range(6):
        curr_date = start_date + timedelta(days=month*30)  # Approximate each month as 30 days
        for account, value in monthly_budget.items():
            if value > 0:
                # If value is positive, it's a debit
                formatted_budget.append((curr_date.strftime('%Y-%m-%d'), account, accounts[account], value, 0))
            elif value < 0:
                # If value is negative, it's a credit
                formatted_budget.append((curr_date.strftime('%Y-%m-%d'), account, accounts[account], 0, -value))
    
    return formatted_budget

# Generate formatted budgets for the entire period
all_formatted_budgets = []
budget_start_date = start_date
for budget in budgets:
    formatted_budget = format_budget_as_gl(budget, budget_start_date)
    all_formatted_budgets.extend(formatted_budget)
    budget_start_date += timedelta(days=6*30)  # Move to the next 6-month period

all_formatted_budgets[:10]  # Displaying the first 10 entries for brevity


[('2021-10-19', 101, 'Cash', 1666.6666666666667, 0),
 ('2021-10-19', 102, 'Accounts Receivable', 833.3333333333334, 0),
 ('2021-10-19', 103, 'Inventory', 416.6666666666667, 0),
 ('2021-10-19', 201, 'Accounts Payable', 0, 333.3333333333333),
 ('2021-10-19', 202, 'Loan Payable', 0, 833.3333333333334),
 ('2021-10-19', 301, "Owner's Equity", 2500.0, 0),
 ('2021-10-19', 401, 'Sales Revenue', 1333.3333333333333, 0),
 ('2021-10-19', 501, 'Cost of Goods Sold', 416.6666666666667, 0),
 ('2021-10-19', 502, 'Rent Expense', 166.66666666666666, 0),
 ('2021-10-19', 503, 'Salary Expense', 250.0, 0)]

In [5]:
def format_budget_with_dates(budget, start_date):
    """Format the budget with start dates for each budgeting period"""
    formatted_budget = []
    for account, value in budget.items():
        formatted_budget.append((start_date.strftime('%Y-%m-%d'), account, accounts[account], value))
    return formatted_budget

# Generate formatted budgets with start dates
all_formatted_budgets_with_dates = []
budget_start_date = start_date
for budget in budgets:
    formatted_budget = format_budget_with_dates(budget, budget_start_date)
    all_formatted_budgets_with_dates.extend(formatted_budget)
    budget_start_date += timedelta(days=6*30)  # Move to the next 6-month period

all_formatted_budgets_with_dates[:10]  # Displaying the first 10 entries for brevity


[('2021-10-19', 101, 'Cash', 10000),
 ('2021-10-19', 102, 'Accounts Receivable', 5000),
 ('2021-10-19', 103, 'Inventory', 2500),
 ('2021-10-19', 201, 'Accounts Payable', -2000),
 ('2021-10-19', 202, 'Loan Payable', -5000),
 ('2021-10-19', 301, "Owner's Equity", 15000),
 ('2021-10-19', 401, 'Sales Revenue', 8000),
 ('2021-10-19', 501, 'Cost of Goods Sold', 2500),
 ('2021-10-19', 502, 'Rent Expense', 1000),
 ('2021-10-19', 503, 'Salary Expense', 1500)]